In [1]:
from google.colab import drive
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load models
model_paths = [
    ("/content/drive/MyDrive/FYP/combine_model/mobilenet/1.keras", (256, 256), "mobilenet"),
    ("/content/drive/MyDrive/FYP/combine_model/nasnetMobile/1.keras", (256, 256), "nasnetmobile"),
    ("/content/drive/MyDrive/FYP/combine_model/vgg/2.keras", (256, 256), "vgg")
]

models = [load_model(path) for path, _, _ in model_paths]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
weights = [0.45, 0.3, 0.25]  # Adjust based on performance mobile: 0.9007633328437805  nas: 0.8702290058135986 vgg: 0.8549618124961853

# Test data path
test_data_dir = "/content/drive/MyDrive/FYP/split/test"

# Create an ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Get true labels
y_true = test_generator.classes

Found 131 images belonging to 2 classes.


In [4]:
# Generate weighted predictions
weighted_predictions = np.zeros((len(y_true), len(test_generator.class_indices)))
for model, weight in zip(models, weights):
    y_pred = model.predict(test_generator)
    weighted_predictions += weight * y_pred

# Final predictions using weighted average
final_predictions = np.argmax(weighted_predictions, axis=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 89s 16s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 30s 5s/step


4/5 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step 

5/5 ━━━━━━━━━━━━━━━━━━━━ 127s 24s/step


In [5]:
# Calculate accuracy
weighted_avg_accuracy = accuracy_score(y_true, final_predictions)
print(f"Accuracy for Weighted Averaging Ensemble: {weighted_avg_accuracy}")

Accuracy for Weighted Averaging Ensemble: 0.9236641221374046
